# Setup

## Imports

In [ ]:
import os
import random
import string
import sys
import time
from IPython.display import clear_output

## Classes

In [ ]:
# Class for origin and destination parameters
class cCoord():
    def __init__(self, y, x):
        self.y = y
        self.x = x

# Chessboard Management

In [ ]:
# FUNCTION: ChessBoardSetup()
# PURPOSE:
#   Creates a new chessboard
# RETURNS:
#   New chessboard

def ChessBoardSetup():
    
    chessBoard = [['r', 't', 'b', 'q', 'k', 'b', 't', 'r'],
                  ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
                  ['.', '.', '.', '.', '.', '.', '.', '.'],
                  ['.', '.', '.', '.', '.', '.', '.', '.'],
                  ['.', '.', '.', '.', '.', '.', '.', '.'],
                  ['.', '.', '.', '.', '.', '.', '.', '.'],
                  ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
                  ['R', 'T', 'B', 'Q', 'K', 'B', 'T', 'R']]

    return chessBoard

def CopyBoard(chessBoard):

    newBoard = ChessBoardSetup();

    for i in range(8):
        for j in range(8):
            newBoard[i][j] = chessBoard[i][j]

    return newBoard;

# FUNCTION: DrawBoard(1)
#   1:      Chessboard
# PURPOSE:
#   Outputs the current board layout

def DrawBoard(chessBoard):

    for i in range(8):
        for j in range(8):
            print(chessBoard[i][j], end=' ')
        print()
    print()

# FUNCTION: IsEnemy(1, 2, 3)
#   1:      Chessboard
#   2:      Coordinates of origin
#   3:      Coordinates of destination
# PURPOSE:
#   Move a piece from the origin to the destination and promote pawns
# RETURNS:
#   Updated chessboard

def MovePiece(chessBoard, origin, dest):
    if (chessBoard[origin.y][origin.x] == 'p' and dest.y == 7):
        chessBoard[origin.y][origin.x] = 'q'
    elif (chessBoard[origin.y][origin.x] == 'P' and dest.y == 0):
        chessBoard[origin.y][origin.x] = 'Q'

    chessBoard[dest.y][dest.x] = chessBoard[origin.y][origin.x]
    chessBoard[origin.y][origin.x] = '.'

    return chessBoard

# FUNCTION: GetPiecesWithLegalMoves(1, 2)
#   1:      Chessboard
#   2:      Team of current player
# PURPOSE:
#   Finds every piece that has a valid move on the chessBoard
# RETURNS:
#   List of pieces that are allowed to move

def GetPiecesWithLegalMoves(chessBoard, sTeam):

    lLegalPieces = []
    
    for i in range(8):
        for j in range(8):
            if (
                (chessBoard[i][j] != '.') and
                (not IsEnemy(sTeam, chessBoard[i][j]))
            ):
                if (len(GetListOfLegalMoves(chessBoard, cCoord(i,j))) > 0):
                    lLegalPieces.append(cCoord(i,j))
                
    return lLegalPieces

# FUNCTION: GetListOfLegalMoves(1, 2)
#   1:      Chessboard
#   2:      Coordinates of origin
# PURPOSE:
#   Finds every place that a piece is allowed to move on the chessBoard
# RETURNS:
#   List of coordinates a piece is allowed to move

def GetListOfLegalMoves(chessBoard, origin):

    lLegalMoves = []

    for i in range(8):
        for j in range(8):
            if (
                IsMoveLegal(CopyBoard(chessBoard), origin, cCoord(i,j)) and
                IsClearPath(CopyBoard(chessBoard), origin, cCoord(i,j)) and
                (not DetectCheck(CopyBoard(chessBoard), origin, cCoord(i,j)))
            ):
                lLegalMoves.append(cCoord(i,j))
            
    return lLegalMoves

# Booleans

## Validation

In [ ]:
# FUNCTION: IsWhite(1)
#   1:      Current piece
# PURPOSE:
#   Check if the current piece is of the white team ('Z')
# RETURNS:
#   True:   Current piece is white ('Z')
#   False:  Current piece is black ('z')

def IsWhite(sATK):
    return True if (sATK <= 'Z' and sATK >= 'A') else False

# FUNCTION: IsEnemy(1, 2)
#   1:      Attacking piece
#   2:      Defending piece
# PURPOSE:
#   Check if attacking and defending pieces are on different teams
# RETURNS:
#   True:   Pieces are on different teams
#   False:  Pieces are the same team

def IsEnemy(sATK, sDEF):
    return True if (IsWhite(sATK) != IsWhite(sDEF)) else False

# FUNCTION: EnemyOf(1)
#   1:      Current piece
# PURPOSE:
#   Return the enemy team color
# RETURNS:
#   Color of enemy team

def EnemyOf(sATK):
    return 'z' if (sATK <= 'Z' and sATK >= 'A') else 'Z'

# FUNCTION: IsDiagonal(1, 2)
#   2:      Coordinates of origin
#   4:      Coordinates of destination
# PURPOSE:
#   Check if coordinates are diagonal
# RETURNS:
#   True:   Coordinates are diagonal
#   False:  Coordinates are not diagonal

def IsDiagonal(origin, dest):
    return True if (abs(origin.x - dest.x) == abs(origin.y - dest.y)) else False

# FUNCTION: [RECURSIVE] IsClearPath(1, 2, 3)
#   1:      COPY of Chessboard
#   2:      Coordinates of origin
#   3:      Coordinates of destination
# PURPOSE:
#   Checks if the path between the origin and destination is clear
# RETURNS:
#   True:   Path is clear
#   False:  Path is not clear

def IsClearPath(chessBoard, origin, dest):

    if (chessBoard[origin.y][origin.x].lower() == 't'): return True

    # if to-square is in the +ve vertical direction from from-square
    if ((origin.x == dest.x) and (origin.y < dest.y)):
        nOrigin = cCoord(origin.y + 1, origin.x)
    # else if to-square is in the -ve vertical direction from from-square
    elif ((origin.x == dest.x) and (origin.y > dest.y)):
        nOrigin = cCoord(origin.y - 1, origin.x)
    # else if to-square is in the +ve horizontal direction from from-square
    elif ((origin.x < dest.x) and (origin.y == dest.y)):
        nOrigin = cCoord(origin.y, origin.x + 1)
    # else if to-square is in the -ve horizontal direction from from-square
    elif ((origin.x > dest.x) and (origin.y == dest.y)):
        nOrigin = cCoord(origin.y, origin.x - 1)
    # else if to-square is in the diagonal direction from from-square
    elif (IsDiagonal(origin, dest)):
        if ((origin.x < dest.x) and (origin.y < dest.y)):
            nOrigin = cCoord(origin.y + 1, origin.x + 1)
        elif ((origin.x < dest.x) and (origin.y > dest.y)):
            nOrigin = cCoord(origin.y - 1, origin.x + 1)
        elif ((origin.x > dest.x) and (origin.y < dest.y)):
            nOrigin = cCoord(origin.y + 1, origin.x - 1)
        elif ((origin.x > dest.x) and (origin.y > dest.y)):
            nOrigin = cCoord(origin.y - 1, origin.x - 1)
    else: return False

    # if destination reached
    if (nOrigin.x == dest.x and nOrigin.y == dest.y): return True

    return False if (chessBoard[nOrigin.y][nOrigin.x] != '.') else IsClearPath(chessBoard, nOrigin, dest)

# FUNCTION: DetectCheck(1, 2, 3)
#   1:      Chessboard
#   2:      Coordinates of origin
#   3:      Coordinates of destination
# PURPOSE:
#   Checks if a hypothetical move puts the current player into check
# RETURNS:
#   True:   King is in check
#   False:  King is not in check

def DetectCheck(chessBoard, origin, dest):

    newBoard = MovePiece(CopyBoard(chessBoard), origin, dest)

    return True if (IsInCheck(newBoard, newBoard[dest.y][dest.x])) else False

# FUNCTION: IsInCheck(1, 2)
#   1:      Chessboard
#   2:      Team of current player
# PURPOSE:
#   Checks if the king can be taken by an enemy piece
# RETURNS:
#   True:   King is in check
#   False:  King is not in check

def IsInCheck(chessBoard, sTeam):

    for i in range(8):
        for j in range(8):
            if (
                chessBoard[i][j].lower() == 'k' and
                (not IsEnemy(sTeam, chessBoard[i][j]))
            ): kLoc = cCoord(i,j)
    
    for i in range(8):
        for j in range(8):
            if (
                chessBoard[i][j] != '.' and
                IsEnemy(chessBoard[kLoc.y][kLoc.x], chessBoard[i][j])
            ):
                if (
                    IsMoveLegal(chessBoard, cCoord(i,j), kLoc) and
                    IsClearPath(CopyBoard(chessBoard), cCoord(i,j), kLoc)
                ): return True

    return False

# FUNCTION: IsCheckmate(1, 2)
#   1:      Chessboard
#   2:      Team of current player
# PURPOSE:
#   Checks if the current player is in checkmate
# RETURNS:
#   True:   Checkmate
#   False:  Not checkmate

def IsCheckmate(chessBoard, sTeam):

    return True if (
        (len(GetPiecesWithLegalMoves(chessBoard, sTeam)) == 0) and
        (IsInCheck(chessBoard, sTeam))
    ) else False

## Rules

In [ ]:
# FUNCTION: IsMoveLegal(1, 2, 3)
#   1:      Chessboard
#   2:      Coordinates of origin
#   3:      Coordinates of destination
# PURPOSE:
#   Checks if a move is valid
# RETURNS:
#   True:   Move is valid
#   False:  Move is invalid

def IsMoveLegal(chessBoard, origin, dest):

    if ((origin.x == dest.x) and (origin.y == dest.y)): return False

    piece = chessBoard[origin.y][origin.x].lower()
    sATK = chessBoard[origin.y][origin.x]
    sDEF = chessBoard[dest.y][dest.x]

    colDiff = origin.x - dest.x
    rowDiff = origin.y - dest.y
    
    # PAWN
    if (piece == 'p'):

        sTeam = 'Z' if (IsWhite(sATK)) else 'z'

        if (sTeam == 'z'):

            if (
                (sDEF == '.') and
                (origin.x == dest.x)
            ):
                if (
                    ((origin.y - dest.y) == -1) or
                    ((origin.y == 1) and (dest.y == 3))
                ): return True

            elif (
                ((rowDiff == -1) and (abs(colDiff) == 1)) and
                (sDEF != '.' and IsEnemy(sATK, sDEF))
            ): return True

        elif (sTeam == 'Z'):

            if (
                (sDEF == '.') and
                (origin.x == dest.x)
            ):
                if (
                    ((origin.y - dest.y) == 1) or
                    ((origin.y == 6) and (dest.y == 4))
                ): return True

            elif (
                ((rowDiff == 1) and (abs(colDiff) == 1)) and
                (sDEF != '.' and IsEnemy(sATK, sDEF))
            ): return True

    elif ((sDEF == '.') or (IsEnemy(sATK, sDEF))):

        # ROOK
        if (piece == 'r'):
            if ((origin.x == dest.x) or (origin.y == dest.y)): return True

        # BISHOP
        elif (piece == 'b'):
            if (IsDiagonal(origin, dest)): return True

        # QUEEN
        elif (piece == 'q'):
            if (
                origin.x == dest.x or
                origin.y == dest.y or
                IsDiagonal(origin, dest)
            ): return True

        # KNIGHT
        elif (piece == 't'):

            if (
                (colDiff == 1 and rowDiff == -2) or
                (colDiff == 2 and rowDiff == -1) or
                (colDiff == 2 and rowDiff == 1) or
                (colDiff == 1 and rowDiff == 2) or
                (colDiff == -1 and rowDiff == -2) or
                (colDiff == -2 and rowDiff == -1) or
                (colDiff == -2 and rowDiff == 1) or
                (colDiff == -1 and rowDiff == 2)
            ): return True

        # KING
        elif (piece == 'k'):

            if (
                (abs(colDiff) == 1 and abs(rowDiff) == 0) or 
                (abs(colDiff) == 0 and abs(rowDiff) == 1) or 
                (abs(colDiff) == 1 and abs(rowDiff) == 1)
            ): return True

    return False

# Artificial Intelligence

## Random AI

In [ ]:
# FUNCTION: GetRandomMove(1, 2)
#   1:      Chessboard
#   2:      Team of current player
# PURPOSE:
#   Creates a random move for a random piece
# RETURNS:
#   Coordinates of random origin and destination

def GetRandomMove(chessBoard, sTeam):

    piece = random.choice(GetPiecesWithLegalMoves(chessBoard, sTeam))
    
    return piece, random.choice(GetListOfLegalMoves(chessBoard, piece))

## MINMAX AI

In [ ]:
# FUNCTION: evl(1)
#   1:      Chessboard
# PURPOSE:
#   Calculates the score for a specified team
# RETURNS:
#   Score of team

def evl(chessBoard):

    score = 0

    for i in range(8):
        for j in range(8):
            if (chessBoard[i][j] != '.'):
                match chessBoard[i][j]:
                    case 'p': score -= 10 * (i+1)
                    case 'P': score += 10 * (9 - (i+1))
                    case 'r': score -= 50
                    case 'R': score += 50
                    case 't': score -= 10
                    case 'T': score += 10
                    case 'b': score -= 30
                    case 'B': score += 30
                    case 'q': score -= 75
                    case 'Q': score += 75
                    case 'k': score -= 0
                    case 'K': score += 0
                    
    return score

# FUNCTION: GetMinMaxMove(1, 2)
#   1:      Chessboard
#   2:      Team of current player
# PURPOSE:
#   Find the move with the MAX score for the MINMAX AI
# RETURNS:
#   Move with the highest MAX score

def GetMinMaxMove(chessBoard, sTeam):

    bestMIN = 1000
    bestMAX = -1000

    bestMAXs = []

    # MAX Player
    pieces = GetPiecesWithLegalMoves(chessBoard, sTeam)

    for origin in pieces:

        destinations = GetListOfLegalMoves(chessBoard, origin)

        for dest in destinations:

            board2 = MovePiece(CopyBoard(chessBoard), origin, dest)

            # MIN Player
            enemyPieces = GetPiecesWithLegalMoves(board2, EnemyOf(sTeam))

            for enemyOrigin in enemyPieces:

                enemyDestinations = GetListOfLegalMoves(board2, enemyOrigin)

                for enemyDest in enemyDestinations:

                    board3 = MovePiece(CopyBoard(board2), enemyOrigin, enemyDest)

                    res = evl(board3)

                    if (res < bestMIN):
                        bestMIN = res

                    if (res < bestMAX): break

                    
            # MIN Player

            if (bestMAX < bestMIN):
                bestMAX = bestMIN
                bestMAXs = []
                bestMAXs.append([origin, dest])

            if (bestMAX == bestMIN):
                bestMAXs.append([origin, dest])

    if (len(bestMAXs) > 1):
        return random.choice(bestMAXs)

# Main Driver

## Testing

## Main

In [ ]:
chessBoard = ChessBoardSetup()

sTeam = 'Z'
turn = 0
N = 500

while ((not IsCheckmate(chessBoard, sTeam)) and (turn < N)):
    clear_output()

    move = GetMinMaxMove(chessBoard, sTeam) if (sTeam == 'Z') else GetRandomMove(chessBoard, sTeam)

    MovePiece(chessBoard, cCoord(move[0].y, move[0].x), cCoord(move[1].y, move[1].x))

    DrawBoard(chessBoard)
    turn+=1
    print("Turn:", turn)
    time.sleep(0.5)

    sTeam = 'z' if (sTeam == 'Z') else 'Z'

if (IsCheckmate(chessBoard, 'z')):
    print("White wins!")
elif (IsCheckmate(chessBoard, 'Z')):
    print("Black wins!")
else:
    print("Stalemate", end='')

    if (evl(chessBoard) > 0):
        print(": White Dominating")
    elif (evl(chessBoard) < 0):
        print(": Black Dominating")

. . . . Q . . . 
. . . . Q . k . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . p 
. . . Q . . . P 
. . B . . K T R 

Turn: 169
White wins!
